### question 10 : sur la spécialisation des pays. 


Le but est d'identifier différents profils de pays :

- ceux qui sont très spécialisés (ie qui ont gagné leurs médailles dans peu de sport différents)
- ceux qui sont plus équilibrés (ie qui ont gagné leurs médailles dans beaucoup de sports différents)


Pour cela on va créer un **ratio de spécialisation**, calculé pour chaque pays comme étant le rapport entre le nombre de médailles obtenues dans le sport le plus représenté et le nombre total de médailles.
Ainsi, un fort ratio indique une forte spécialisation.

In [6]:
# on importe le package
import pandas as pd

# on charge les données
athletes = pd.read_csv('athlete_events.csv')
noc = pd.read_csv('noc_regions.csv')

# fusion des deux tableaux
data = pd.merge(athletes, noc, how='left', on='NOC')

# on ne garde que les places avec médailles
data_medals = data.dropna(subset = ['Medal'])

# on calcule le nombre de médailles par sport, par pays
grouped = data_medals.groupby(['NOC','Sport']).size().reset_index(name='podium_count')

# on sélectionne le sport avec le plus de médailles pour chaque pays
max_podium = grouped.groupby('NOC')['podium_count'].max().reset_index(name="max_podium")

# on calcule le nombre total de médailles par pays
total_podium = grouped.groupby('NOC')['podium_count'].sum().reset_index(name='total_podium')

#on fusionne dans un tableau
specialisation = pd.merge(max_podium, total_podium, on='NOC')
specialisation = specialisation.merge(noc[['NOC','region']], on='NOC', how='left')

# on calcule le ratio de spécialisation de chaque pays. 
specialisation['ratio_specialisation'] = specialisation['max_podium'] / specialisation['total_podium']

# Réorganiser pour placer le nom du pays en première colonne et renommer si besoin
specialisation = specialisation[['region', 'max_podium', 'total_podium', 'ratio_specialisation']]
specialisation.rename(columns={'region': 'Country'}, inplace=True)

In [7]:
#ces réglages permettent d'afficher toutes les lignes des tableaux (pour chercher des valeurs pertinentes)
# si on veut afficher le tableau complet, on enlèvera le .head()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Pays avec le plus faible taux de spécialisation (remarques en bas du tableau)
spe_asc = specialisation.sort_values(by='ratio_specialisation', ascending = True).head(20)
spe_asc

,Country,max_podium,total_podium,ratio_specialisation
112,Russia,103,1165,0.088412
40,Russia,30,279,0.107527
49,Germany,236,2165,0.109007
22,China,109,989,0.110212
139,Russia,288,2503,0.115062
12,Belgium,54,468,0.115385
138,Ukraine,23,199,0.115578
44,Germany,71,586,0.121160
120,Switzerland,92,691,0.133140
123,Sweden,217,1536,0.141276


In [8]:
# afficher le ratio de spécialisation des pays avec le plus de médailles.

tab2 = specialisation.sort_values(by='total_podium', ascending = False).head(15)
tab2

#rq : pour voir le tableau en entier, il suffit d'enlever .head()

,Country,max_podium,total_podium,ratio_specialisation
141,USA,1080,5637,0.191591
139,Russia,288,2503,0.115062
49,Germany,236,2165,0.109007
46,UK,338,2068,0.163443
43,France,310,1777,0.174451
67,Italy,359,1637,0.219304
123,Sweden,217,1536,0.141276
20,Canada,348,1352,0.257396
6,Australia,401,1320,0.303788
112,Russia,103,1165,0.088412


### 🔭 Observations

1) souvent, un taux de spécialisation proche de 1 signifie que le pays a gagné une ou peu de médailles. 

2) cependant, certains pays ont un très fort taux de spécialisation tout en ayant gagné beaucoup de médailles. C'est le cas de la Jamaïque . Cela montre leur domination dans un sport particulier. 

* L'Éthiopie a 53 médailles, toutes dans un seul sport
* La Jamaïque a 157 médailles au total donc 156 dans un seul sport

3) Il serait intéressant ensuite d'identifier quels sont ces sports dans lesquels ces pays sont très spécialisés. 


### 📝 Remarques

1) le ratio de spécialisation en lui-même ne dit pas tout puisqu'un pays peu avoir un faible taux de spécilisation mais peu de médailles. 

2) Ainsi on pourrait créer un indicateur de performance général qui valoriserait à la fois le nombre de médailles gagnées ET le fait d'être peu spécialisé. 

3) performance = {nombre total de médailles gagnées} x [1 - ratio_specialisation]

4) Un pays avec beaucoup de médailles et un faible ratio aura un score élevé. Un pays avec beaucoup de médailles mais un ratio de spécialisation élevé aura un score moyen. 

### 🦺 Piste d'approfondissement

(en cours)
afficher sur une carte choroplèthe le ratio de spécialisation de chaque pays, en utilisant géopandas et potentiellement d'autres packages
   
intuitions (après les premières observations): 
- l'Europe aura probablement un ratio de spécialisation plus faible
- l'Afrique aura un ratio de spécialisation plus élevé (dans la course, l'athlétisme)